In [ ]:
# default_exp sampler.base_sampler

# Sampler base class

> Super class for all sampler objects

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
from qsam.datatypes.counttree import CountTree, Variable, Constant
from qsam.datatypes.protocol import iterate
from qsam.datatypes.circuit import unpack

import itertools as it
from tqdm.auto import tqdm

In [ ]:
#export

class BaseSampler:
    
    def __init__(self, protocol, simulator, err_mod, *args, **kwargs):
        
        self.protocol = protocol
        self.simulator = simulator
        
        min_path_weight = 2 if protocol.fault_tolerant else 1
        self.tree = CountTree(min_path_weight=min_path_weight)
        
        fault_gen.configure(self.protocol._circuits)
        self.w_vecs = protocol_weight_vectors(fault_gen.partitions)
        self.fault_gen = fault_gen
        
        self.n_qubits = len(set(q for c in protocol._circuits.values() for q in unpack(c)))
        self.setup(*args, **kwargs)
        
    def setup(self, *args, **kwargs):
        """Pass-through to child class."""
        pass
        
    def sample(self, callbacks, *args, **kwargs):
        assert getattr(self, "loop_fn")
        
        callbacks.on_protocol_begin()
        sim = self.simulator(self.n_qubits)
        p_it = iterate(self.protocol)
        node = next(p_it)
        tree_node = None

        while True:
            callbacks.on_circuit_begin()

            tree_node = self.tree.add(name=node, parent=tree_node, nodetype=Variable)
            tree_node.counts += 1

            if node == None:
                fail = 0; break
            elif not self.protocol.out_edges(node):
                tree_node.marked = True
                fail = 1; break

            circuit_hash, circuit = self.protocol.circuit_from_node(node)
            if not circuit._noisy or circuit_hash not in self.protocol._circuits.keys():
                msmt = sim.run(circuit)
            else:
                # w_vec, w_idx, faults = self.loop_fn( kwargs | locals() )
                loop_out = self.loop_fn( kwargs | locals() )
                tree_node = self.tree.add(name=loop_out["w_vec"], parent=tree_node, nodetype=Constant, c_hash=circuit_hash, w_idx=loop_out["w_idx"],
                                         is_deterministic=True if circuit._ff_deterministic and not any(loop_out["w_vec"]) else False)
                tree_node.counts += 1
                fault_circuit = self.fault_gen.gen_circuit(len(circuit), loop_out["faults"])
                msmt = sim.run(circuit, fault_circuit)

            _node = node
            node = p_it.send(msmt)

            callbacks.on_circuit_end(locals())

        callbacks.on_protocol_end()
        
        return fail

## Helper methods

In [ ]:
#export
def circuit_partitions(circ, partition_names):
    return [partition(circ, GATE_GROUPS[name]) for name in partition_names]

def protocol_partitions(circuits_dict, partition_names):
    return {c_hash: circuit_partitions(circ, partition_names)
           for c_hash, circ in circuits_dict.items() if circ._noisy}

In [ ]:
#export
def circuit_weight_vectors(w_max, w_exclude={}):
    w_exclude = set((w,) if isinstance(w, int) else w for w in w_exclude)
    w_maxs = [w_max] if isinstance(w_max, int) else w_max

    w_upto_w_maxs = [tuple(range(w_max+1)) for w_max in w_maxs]
    w_vecs = list(it.product( *w_upto_w_maxs ))
    filtered_w_vecs = [w for w in w_vecs if w not in w_exclude]
    return filtered_w_vecs

def protocol_weight_vectors(partition_dict):
    return {c_hash: circuit_weight_vectors([len(p) for p in partitions]) 
            for c_hash, partitions in partition_dict.items()}